In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import time
import tensorflow as tf
import shap
import numpy as np

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
df = pd.read_csv('UNSW_NB15_balanced_smote_lime.csv')
columns = df.columns.tolist()

In [3]:
#read kdd_processed_bin_top15.csv
df_bin = pd.read_csv('unsw_shap_top_15.csv')

# get the names of the columns in a list
columns_bin = df_bin.columns.tolist()

In [4]:
#combine the two lists of columns without duplicates
columns = list(set(columns + columns_bin))
print(columns)
#print length of columns
print(len(columns))

['proto_sep', 'proto_fc', 'proto_iso-ip', 'ct_srv_dst', 'proto_udp', 'ct_state_ttl', 'rate', 'dmean', 'proto_tcp', 'proto_ax.25', 'proto_pnni', 'swin', 'service_None', 'proto_mfe-nsp', 'sload', 'proto_visa', 'dttl', 'dwin', 'dload', 'is_sm_ips_ports', 'sttl', 'label', 'proto_prm', 'ct_dst_src_ltm', 'proto_tcf', 'proto_tlsp', 'proto_ipv6-opts', 'ackdat', 'proto_sctp', 'state_CLO']
30


In [5]:
data = pd.read_csv('UNSW_NB15_balanced_smote.csv')

In [6]:
data = data.drop(data.columns.difference(columns), axis=1)
data

,rate,sttl,dttl,sload,dload,swin,dwin,ackdat,dmean,ct_state_ttl,...,proto_prm,proto_sctp,proto_sep,proto_tcf,proto_tcp,proto_tlsp,proto_udp,proto_visa,service_None,state_CLO
0,0.000462,2.458823,2.252534,0.000076,0.003522,2.002085,2.004241,0.000000,0.169264,0.000000,...,False,False,False,False,True,False,False,False,True,False
1,0.000489,0.604948,2.234797,0.000045,0.208746,2.002085,2.004241,0.000000,4.353636,1.007762,...,False,False,False,False,True,False,False,False,True,False
2,0.000088,0.604948,2.234797,0.000008,0.025257,2.002085,2.004241,1.101997,3.243577,1.007762,...,False,False,False,False,True,False,False,False,True,False
3,0.000085,0.604948,2.234797,0.000015,0.001392,2.002085,2.004241,0.000000,0.251928,1.007762,...,False,False,False,False,True,False,False,False,False,False
4,0.000208,2.478337,2.234797,0.000046,0.001653,2.002085,2.004241,1.250455,0.177137,1.007762,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929995,0.000349,2.478337,2.234797,0.000462,0.002584,2.002085,2.004241,1.103808,0.177137,1.007762,...,False,False,False,False,True,False,False,False,False,False
929996,0.000242,2.478337,2.234797,0.000082,0.001919,2.002085,2.004241,0.850329,0.177137,1.007762,...,False,False,False,False,True,False,False,False,False,False
929997,0.000371,2.478337,2.234797,0.000166,0.002947,2.002085,2.004241,0.548006,0.177137,1.007762,...,False,False,False,False,True,False,False,False,False,False
929998,0.000540,2.478337,2.234797,0.000030,0.315734,2.002085,2.004241,1.582174,5.182381,1.007762,...,False,False,False,False,True,False,False,False,False,False


In [7]:
# train an lstm model on the data
X = data.drop(['label'], axis=1)
Y = data['label']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

# define the model and add recurrent dropout
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0929 - accuracy: 0.9702 - val_loss: 0.0837 - val_accuracy: 0.9704
Epoch 2/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0834 - accuracy: 0.9708 - val_loss: 0.0842 - val_accuracy: 0.9701
Epoch 3/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0810 - accuracy: 0.9709 - val_loss: 0.0810 - val_accuracy: 0.9703
Epoch 4/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0799 - accuracy: 0.9709 - val_loss: 0.0805 - val_accuracy: 0.9704
Epoch 5/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0792 - accuracy: 0.9709 - val_loss: 0.0839 - val_accuracy: 0.9704
Epoch 6/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0789 - accuracy: 0.9710 - val_loss: 0.0773 - val_accuracy: 0.9706
Epoch 7/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0785 - accuracy: 0.9710 - val_loss: 0

In [8]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred, digits=4))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print("Time taken to predict: ", end_time-start_time)

5813/5813 [==============================] - 3s 526us/step
Classification Report
              precision    recall  f1-score   support

           0     0.9952    0.7123    0.8303     18731
           1     0.9688    0.9996    0.9840    167269

    accuracy                         0.9707    186000
   macro avg     0.9820    0.8560    0.9071    186000
weighted avg     0.9714    0.9707    0.9685    186000

Accuracy:
0.9706774193548388
Time taken to predict:  4.097585916519165
